In [ ]:
!!pip install -q MOABB

In [ ]:
!pip install pywd

In [ ]:
!pip install mlflow

In [4]:
import numpy as np
import scipy.signal
import scipy.io as sio
import matplotlib
from matplotlib import pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
import mne
from pywt import wavedec 

import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score
import mlflow
import mlflow.sklearn

In [5]:
import moabb
from moabb.datasets import EPFLP300, utils
from moabb.evaluations import CrossSessionEvaluation
from moabb.paradigms import LeftRightImagery
from moabb.pipelines.features import LogVariance
from mne.filter import construct_iir_filter
moabb.set_log_level("info")


import math
from sklearn.metrics import confusion_matrix

In [6]:
dataset=EPFLP300()

In [7]:
def getData(subjectNumbers,dataset=EPFLP300()):
  return dataset.get_data(subjectNumbers)

In [8]:
def getSubjectData(subjectNumber,session_number,run_number):
  session = f'session_{session_number}'
  run = f'run_{run_number}'
  return getData([subjectNumber])[subjectNumber][session][run]

In [121]:
def filterAndResampleData(data,seglen,lp=20,hp=0.1,sf=2048,resample=64):
  iir_params = dict(order=6, ftype='butter', output='sos')  
  iir_params = construct_iir_filter(iir_params, [hp,lp], None, sf, 'bandpass', return_copy=False)
  data.filter(method='iir',iir_params=iir_params,h_freq=hp,l_freq=lp)
  return data.resample(math.ceil(((sf*64)/seglen)))

In [106]:
def getTargetData(df):
  Target = df[df['STI']==2].index.values
  val = np.full(len(Target),2)
  targetDF= pd.DataFrame(Target)
  targetDF["value"]=val
  return targetDF

In [107]:
def getNonTargetData(df):
  nonTarget = df[df['STI']==1].index.values
  value = np.full(len(nonTarget),1)
  nontargetDF= pd.DataFrame(nonTarget)
  nontargetDF["value"]=value
  return nontargetDF

In [109]:
def getTottalData(df):
  targetDF = getTargetData(df)
  nontargetDF = getNonTargetData(df)
  totaldf = nontargetDF.append(targetDF,ignore_index=True) 
  TargetNnonTarget = totaldf.sort_values(by=[0],ignore_index=True)
  return TargetNnonTarget

In [122]:
def scaleAndExtractFeatures(segment):
  scaler = MinMaxScaler(feature_range = (-1,1))
  df = segment.to_data_frame()
  columns = ['time', 'MA1', 'MA2' , 'STI']
  df.drop(columns,axis=1,inplace=True)
  featureVector = []
  for val in df.columns.values:
    vals=df[val].values.reshape(-1, 1)
    scaled = scaler.fit_transform(vals)
    # print("rum number: ",runNum,"\nsegment number",segindx)
    col= scaled.reshape(64,)
    cA2, cD2, cD1 = wavedec(col,'db4',level=2)
    featureVector.extend(cA2)
    featureVector.extend(cD2)
    featureVector.extend(cD1)
  return featureVector

In [123]:
def divideSingleRun(df, data):
  TargetNnonTarget= getTottalData(df)
  segments=[]
  output=[]
  length=len(TargetNnonTarget)
  for index, row in TargetNnonTarget.iterrows():
    if(index==length-1):
      Segment = df.iloc[TargetNnonTarget[0].iloc[index]:]
    else:
      Segment = df.iloc[TargetNnonTarget[0].iloc[index]:TargetNnonTarget[0].iloc[index]+2048]

    # crop segment from RawArray
    d = data.copy()
    Segment.reset_index(inplace=True)
    tmax= Segment['time'][len(Segment)-1]
    tmin= Segment['time'][0]
    seg=d.crop(tmin=tmin, tmax=tmax)

    # Filter and resample
    filtered = filterAndResampleData(seg,len(Segment))

    # Scale and extract features
    if(len(filtered.to_data_frame())!=64):
      continue
    featureVector= scaleAndExtractFeatures(filtered)

    segments.append(featureVector)
    output.append(TargetNnonTarget['value'].iloc[index])
  return segments, output

In [112]:
def extractData(subjectNumber):
  rawDataTrain=[]
  rawDataTest=[]
  for sessionNumber in range(1,4):
    for runNumber in range(1,7):
      data = getSubjectData(subjectNumber,sessionNumber,runNumber)
      rawDataTrain.append(data)
  for runNumber in range(1,7):
    d= getSubjectData(subjectNumber,4,runNumber)
    rawDataTest.append(d)
  return rawDataTrain,rawDataTest

In [ ]:
rawTrain, rawTest =extractData(1)

In [113]:
def Preprocess(rawTrain, rawTest):
  x_train=[]
  y_train=[]
  for indx in range(18):
    df = rawTrain[indx].to_data_frame()
    segments, target = divideSingleRun(df, rawTrain[indx])
    x_train+=segments
    y_train+=target
  x_test =[]
  y_test =[]
  for indx in range(6):
      df=rawTest[indx].to_data_frame()
      segments, target = divideSingleRun(df, rawTest[indx])
      x_test+=segments
      y_test+=target
  return x_train,y_train, x_test ,y_test

In [124]:
x_train,y_train, x_test ,y_test=Preprocess(rawTrain, rawTest)

In [99]:
def SVM(subjectNumber,x_train,y_train, x_test ,y_test):
  print("preparing Subject ",subjectNumber , " Data.......")
  clf = svm.SVC()
  clf.fit(x_train,y_train)
  predictions =clf.predict(x_test)
  print(subjectNumber , "accuracy score " , accuracy_score(predictions , y_test))
  
  cm = confusion_matrix(y_test,predictions,labels=[2,1])
  print(cm)
  print("______________________________________________")

This result with filtering to (0.1, 20) and with resampling to 64

In [125]:
SVM(1,x_train,y_train, x_test ,y_test)

preparing Subject  1  Data.......
1 accuracy score  0.8611449451887941
[[ 26 110]
 [  4 681]]
______________________________________________
